In [1]:
# from src.acoustic_analyser import frame
# import numpy as np

# test_frame = frame.from_file(
#     member_file="test_member.json", constraint_file="test_constraint.csv", debug=False
# )

# free_end = test_frame.fixed_end(member_id=0)
# free_end = test_frame.free_end(member_id=1)

In [2]:
from src.acoustic_analyser import frame
import numpy as np


def printa(array):
    for i in array:
        print(i)


test_frame = frame()

test_frame.add_member(
    length=0.5, density=7800, youngs_modulus=206e9, height=1.27e-2, id=0
)
test_frame.add_member(
    length=0.5, density=7800, youngs_modulus=206e9, height=1.27e-2, id=1
)

free_end = test_frame.fixed_end(member_id=0)
joint = test_frame.two_member_joint(member_1_id=0, member_2_id=1, theta=90)
free_end = test_frame.free_end(member_id=1)

In [3]:
np.linalg.det(test_frame.get_equation_matrix(w=2 * np.pi * 13.742))

(1.3689194733634974e-06-7.831950446103157e-06j)

In [4]:
# _ = test_frame.get_frequency_graph(lower_limit=1, upper_limit=100, step_size=2)

In [5]:
# test_frame.get_natural_frequency_newton(initial_guess=10)

In [6]:
# test_frame.get_natural_frequency_bisect(lower_limit=50, upper_limit=55)

In [7]:
subs_dict = test_frame.get_params_solution(natural_freq=13.742)

In [15]:
from src.modules.bc import fixed_end, free_end
from matplotlib import pyplot as plt

step_size = 0.1
freq = 13.742
constraint_curr = test_frame.constraints[0]
member_curr = constraint_curr.members[0]
offset = np.array([0, 0])
angle = 0
Handle = True
mode_shape = [np.array([[0], [0]]).reshape(1, 1, 2)]
members_completed = set()
constraints_completed = set()

while Handle:
    members_completed.add(member_curr.id)
    constraints_completed.add(constraint_curr.id)

    x = np.append(
        np.arange(step_size, member_curr.length, step_size), member_curr.length
    )
    v1, u1 = member_curr.get_deformation(
        w=2 * np.pi * freq, lengths=x, id=constraint_curr.id, subs_dict=subs_dict
    )
    print(v1[1])
    print(u1[1])
    print(x[1])
    break  # Works till here
    x_deformed = np.real(v1) + x
    y_deformed = np.real(u1)
    points_deformed = np.stack([x_deformed, y_deformed]).T.reshape(len(x), 1, 2)

    Rotation_matrix = np.array(
        [[np.cos(angle), -np.sin(angle)], [np.sin(angle), np.cos(angle)]]
    )
    rotated = np.matmul(points_deformed, np.stack([Rotation_matrix] * len(x_deformed)))
    rotated_translated = rotated + offset
    mode_shape.append(rotated_translated)

    constraint_ids = member_curr.constraint_ids
    if constraint_ids[0] not in constraints_completed:
        constraint_curr = test_frame.constraints[constraint_ids[0]]
    else:
        constraint_curr = test_frame.constraints[constraint_ids[1]]

    members = constraint_curr.members  # Will have to change for a 3 member joint

    if members[0].id not in members_completed:
        member_curr = members[0]
    elif len(members) > 1 and members[1] not in members_completed:
        member_curr = members[1]
    else:
        Handle = False

    angle = -constraint_curr.theta
    offset = rotated_translated[-1]
mode_shape = np.concatenate(mode_shape)
x = mode_shape[:, :, 0]
y = mode_shape[:, :, 1]
plt.plot(x, y)

[[0.0352872-1.79613233e-07j]]
[[-5.10829968e-06-1.62510616e-11j]]
0.2


ValueError: all input arrays must have the same shape

In [9]:
print("Essential till mode shape verified")
constraint_curr = test_frame.constraints[0]
member_curr = constraint_curr.members[0]
from sympy import Matrix, E, I

points = [0, 0.1, 0.2, 0.3, 0.4, 0.5]


def get_propagation_matrix(obj, w: float, length: float):
    C = (obj.youngs_modulus / obj.density) ** 0.5
    K = (obj.inertia / obj.cross_section_area) ** 0.5

    alpha = (w * K / C) ** 0.5
    beta = w * K / C
    L_bar = length / K

    propagation_matrix = Matrix(
        [
            [E ** (-I * alpha * L_bar), 0, 0],
            [0, E ** (-alpha * L_bar), 0],
            [0, 0, E ** (-I * beta * L_bar)],
        ]
    )
    return propagation_matrix


for i in points:
    w = 2 * np.pi * 13.742
    propagation_matrix_subs = get_propagation_matrix(member_curr, w=w, length=i)
    propagation_matrix_inv_subs = propagation_matrix_subs.inv()
    a_plus_subs = member_curr.a_plus.subs(subs_dict)
    a_minus_subs = member_curr.a_minus.subs(subs_dict)

    v = (
        Matrix([1, 1, 0]).T * propagation_matrix_subs * a_plus_subs
        + Matrix([1, 1, 0]).T * propagation_matrix_inv_subs * a_minus_subs
    ).evalf()
    u = (
        Matrix([0, 0, 1]).T * propagation_matrix_subs * a_plus_subs
        + Matrix([0, 0, 1]).T * propagation_matrix_inv_subs * a_minus_subs
    ).evalf()
    print(i)
    print((Matrix([1, 1, 0]).T * propagation_matrix_subs * a_plus_subs).evalf()[0])
    print(v[0])
    print(u[0])
    print("---------------------------------")

Essential till mode shape verified
0
0.0703000196537892 + 0.0702994603253966*I
1.46224217442703e-7 - 1.53756644222236e-7*I
-1.62123414411793e-9 + 4.06859813851732e-11*I
---------------------------------
0.1
0.0536331880427632 + 0.0912715787099289*I
0.0092809067179299 - 1.382418495783e-7*I
-2.55496406242653e-6 + 1.2217477134e-11*I
---------------------------------
0.2
0.0475829222318819 + 0.108076804228468*I
0.0352872004762986 - 1.796132334108e-7*I
-5.10829967844353e-6 - 1.62510616044e-11*I
---------------------------------
0.3
0.0499076378851822 + 0.119947934893222*I
0.0752889055389118 - 2.6801683448333e-7*I
-7.66162087452652e-6 - 4.47195544689e-11*I
---------------------------------
0.4
0.0584686155921051 + 0.126343023940374*I
0.12663490880286 - 3.939923771758e-7*I
-1.02149204430477e-5 - 7.31879210972e-11*I
---------------------------------
0.5
0.0712341345985477 + 0.126970121815278*I
0.186836189409652 - 5.4865562483936e-7*I
-1.27681911764401e-5 - 1.01656081127e-10*I
-----------------

In [10]:
propagation_matrix_subs = member_curr.get_propagation_matrix(
    w=2 * np.pi * 13.742, lengths=[0.1]
)
propagation_matrix_inv_subs = np.linalg.inv(propagation_matrix_subs)
a_plus_subs = np.matrix(member_curr.a_plus.subs(subs_dict), dtype=complex)
a_minus_subs = np.matrix(member_curr.a_minus.subs(subs_dict), dtype=complex)
v = (
    np.matmul(np.array([1, 1, 0]), propagation_matrix_subs) * a_plus_subs
    + np.matmul(np.array([1, 1, 0]), propagation_matrix_inv_subs) * a_minus_subs
)
u = (
    np.matmul(np.array([0, 0, 1]), propagation_matrix_subs) * a_plus_subs
    + np.matmul(np.array([0, 0, 1]), propagation_matrix_inv_subs) * a_minus_subs
)
np.matmul(np.array([1, 1, 0]), propagation_matrix_subs) * a_plus_subs

matrix([[0.05363319+0.09127158j]])

In [11]:
u

matrix([[-2.55496406e-06+1.22174772e-11j]])

In [12]:
v

matrix([[0.00928091-1.3824185e-07j]])